In [1]:
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz

In [2]:
# Set up dummy data for BGG
BGG = pd.DataFrame([{
    "names": "Gloomhaven",
    "id": 123,
    "rating": "9.5",
    "num_reviewers": 644,
},{
    "names": "Gloomhaven: The Best Expansion Set",
    "id": 124,
    "rating": "9.8",
    "num_reviewers": 124,
},{
    "names": "Not Kickstarter",
    "id": 200,
    "rating": "4.5",
    "num_reviewers": 23,
},{
    "names": "Election",
    "id": 344,
    "rating": "8.5",
    "num_reviewers": 134,
}, {
    "names": "Pandemic",
    "id": 544,
    "rating": "8.5",
    "num_reviewers": 134,
}])
BGG

,names,id,rating,num_reviewers
0,Gloomhaven,123,9.5,644
1,Gloomhaven: The Best Expansion Set,124,9.8,124
2,Not Kickstarter,200,4.5,23
3,Election,344,8.5,134
4,Pandemic,544,8.5,134


In [3]:
# Set up dummy data for kick_BG
kick_BG = pd.DataFrame([{
    "name": "Gloomhaven: The Best Game Ever",
    "status": "success",
    "usd_pledged": 502312,
    "kick_id": 123434,
},{
    "name": "Election",
    "status": "failure",
    "usd_pledged": 5,
    "kick_id": 234123,
},{
    "name": "Some Obscure Kickstarter game with no BGG page",
    "status": "success",
    "usd_pledged": 2132,
    "kick_id": 523434,
}])
kick_BG

,name,status,usd_pledged,kick_id
0,Gloomhaven: The Best Game Ever,success,502312,123434
1,Election,failure,5,234123
2,Some Obscure Kickstarter game with no BGG page,success,2132,523434


In [4]:
"""
Given a name (e.g. "Gloomhaven: Best Board Game Ever")
and a list of other names (e.g. ["Gloomhaven", "Pandemic"]) (this list will likely be a whole column of names, e.g. BGG["names"]),
Return the name in the list which has the highest fuzz.partial_ratio,
but the partial_ratio has to be over a threshold (default: 70).
"""
def best_partial_ratio(name, other_names, threshold=90):
    scores = [(other_name, fuzz.partial_ratio(name, other_name)) for other_name in other_names]
    best = max(scores, key=lambda p: p[1])
    if best[1] > threshold:
        return best
    return None, 0


In [5]:
# Example usage
print(best_partial_ratio("Gloomhaven: the best board game of all time", BGG["names"]))
print(best_partial_ratio("Election", BGG["names"]))
print(best_partial_ratio("Some Kickstarter Game with no BGG Page", BGG["names"]))


('Gloomhaven', 100)
('Election', 100)
(None, 0)


In [6]:
# Add a new column to kick_BG, indicating which name on BGG it matched with most closely
kick_BG["BGG_match"] = kick_BG["name"].apply(lambda x: best_partial_ratio(x, BGG["names"])[0])

In [7]:
kick_BG

,name,status,usd_pledged,kick_id,BGG_match
0,Gloomhaven: The Best Game Ever,success,502312,123434,Gloomhaven
1,Election,failure,5,234123,Election
2,Some Obscure Kickstarter game with no BGG page,success,2132,523434,None


In [8]:
# Merge (using exact match) on kick_BG.BGG_match and BGG.names
merged = pd.merge(kick_BG, BGG, how="inner", left_on="BGG_match", right_on="names")
# Delete temporary column BGG_match
merged = merged.drop("BGG_match", axis=1)

In [9]:
merged # now contains data from both BGG and kick_BG

,name,status,usd_pledged,kick_id,names,id,rating,num_reviewers
0,Gloomhaven: The Best Game Ever,success,502312,123434,Gloomhaven,123,9.5,644
1,Election,failure,5,234123,Election,344,8.5,134
